In [1]:
from utils.infer import *
import pandas as pd
from  utils.eval import *
from utils.chat import *
from utils.data import *

%reload_ext autoreload
%autoreload 2

/Users/paul/anaconda3/envs/ClinicalNotes/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Combining inferences

In [2]:
inference_paths = "../data/inference"
output_path = "../data/combined_inferences.jsonl"

combined = combine(inference_paths, output_path)

### Building evaluation inputs

In [3]:
note_models = [model for model in MODELS_TO_MODE.keys() if 'summarizer' not in MODELS_TO_MODE[model] ]

note_models.remove('llama-2-7b-chat')
note_models.remove('llama-2-13b-chat')

summarizer_models = [model for model in MODELS_TO_MODE.keys() if 'summarizer' in MODELS_TO_MODE[model]]

all_models = note_models + summarizer_models
print(all_models)


['meditron-7b-direct-trunc', 'meditron-13b-direct-trunc', 'meditron-7b-generator', 'meditron-13b-generator', 'mistral-7b-direct', 'gpt3-direct', 'gpt3-generator-gpt', 'gpt3-generator-7b', 'gpt3-generator-13b', 'meditron-7b-generator-gold', 'meditron-13b-generator-gold', 'meditron-7b-summarizer', 'meditron-13b-summarizer']


In [4]:
build_evaluation_inputs(output_path, models = all_models)

[('meditron-7b-direct-trunc', 'meditron-13b-direct-trunc'), ('meditron-7b-direct-trunc', 'meditron-7b-generator'), ('meditron-7b-direct-trunc', 'meditron-13b-generator'), ('meditron-7b-direct-trunc', 'mistral-7b-direct'), ('meditron-7b-direct-trunc', 'gpt3-direct'), ('meditron-7b-direct-trunc', 'gpt3-generator-gpt'), ('meditron-7b-direct-trunc', 'gpt3-generator-7b'), ('meditron-7b-direct-trunc', 'gpt3-generator-13b'), ('meditron-7b-direct-trunc', 'meditron-7b-generator-gold'), ('meditron-7b-direct-trunc', 'meditron-13b-generator-gold'), ('meditron-7b-direct-trunc', 'gold'), ('meditron-13b-direct-trunc', 'meditron-7b-generator'), ('meditron-13b-direct-trunc', 'meditron-13b-generator'), ('meditron-13b-direct-trunc', 'mistral-7b-direct'), ('meditron-13b-direct-trunc', 'gpt3-direct'), ('meditron-13b-direct-trunc', 'gpt3-generator-gpt'), ('meditron-13b-direct-trunc', 'gpt3-generator-7b'), ('meditron-13b-direct-trunc', 'gpt3-generator-13b'), ('meditron-13b-direct-trunc', 'meditron-7b-generat

###  [OLD] Evaluation GPT 3 direct

In [3]:
gpt_3_eval_res = note_evaluation('direct-gpt')

Evaluation directory exists, loading...
Progress: {'create eval directory': 'done', 'scores': 'done', 'bleu': 'done', 'rouge': 'done', 'bert': 'done', 'gpt_score': 'done'}


In [4]:
display(gpt_3_eval_res.sort_values(by='aggregated_score', ascending=False))

,idx,gold,pred,model_name,bleu,bert,gpt_score,rouge1,rouge2,rougeL,rougeLsum,aggregated_score
767,56341,A 46-year-old woman presented with intermitten...,Patient presented with a two-month history of ...,direct-gpt,0.384352,0.947115,8,0.938389,0.852381,0.928910,0.928910,1.406937
307,125854,"A healthy, athletic 19-year-old African Americ...",Patient presented for a surgical evaluation of...,direct-gpt,0.256832,0.885681,9,0.904645,0.796069,0.860636,0.860636,1.183339
213,165122,"A 19-year-old, single, unemployed, Caucasian f...",Patient presented with multiple pigmented annu...,direct-gpt,0.309861,0.901271,9,0.864450,0.771208,0.843990,0.843990,1.166625
534,146809,"The present case is about a 48-year-old, femal...",Patient presented with complaints of fullness ...,direct-gpt,0.598819,0.882541,9,0.806604,0.658768,0.764151,0.764151,1.086166
417,117607,The author reports a case of 30-year-old male ...,Patient presents to the physiotherapy outpatie...,direct-gpt,0.140051,0.879066,9,0.840295,0.735802,0.830467,0.830467,1.056809
...,...,...,...,...,...,...,...,...,...,...,...,...
518,170283,A 70-year-old nonverbal female presented to th...,Patient presented to the emergency department ...,direct-gpt,0.182045,0.809504,8,0.340580,0.167883,0.231884,0.231884,0.229316
341,163372,A 23-year-old male patient presented to us in ...,Patient presented with a deformity of the back...,direct-gpt,0.050793,0.915659,2,0.395480,0.187500,0.305085,0.305085,0.221921
270,162274,"A 26-year-old man, educated in an engineering ...",Patient presented with a chief complaint of co...,direct-gpt,0.165997,0.851599,7,0.347561,0.085890,0.250000,0.250000,0.218151
755,15191,A 55-year-old male who had been diagnosed with...,Patient presented with a two-week history of i...,direct-gpt,0.237331,0.890861,2,0.385852,0.213592,0.327974,0.327974,0.214040


### Evaluation meditron-7b-summarizer

In [51]:
summarizer7_eval_by_sample , summarizer7_eval_by_key = summary_evaluation('meditron-7b-summarizer')

Running summary evaluation with score types: ['bleu', 'rouge', 'bert', 'gpt_score']
Loading data...
Evaluation directory exists, loading...
Progress: {'create eval directory': 'done', 'flatten and match dicts': 'done', 'clean dicts and counts': 'done', 'summary_statistics': 'done', 'scores': 'done', 'pairs_idx': 'done', 'eval_by_sample': 'tbd', 'eval_by_key': 'done', 'bleu': 'done', 'rouge': 'done', 'bert': 'done', 'gpt_score': 'done'}


In [68]:
display(summarizer7_eval_by_sample.sort_values(by='aggregated_score', ascending=False))
top_10_idxs = summarizer7_eval_by_sample.sort_values(by='aggregated_score', ascending=False)['idx'][:10]
display(summarizer7_eval_by_key)

,idx,gold,pred,mean_bleu,mean_bert,mean_gpt_score,mean_rouge1,mean_rouge2,mean_rougeL,mean_rougeLsum,aggregated_score,missing_keys_count,extra_keys_count,common_none_count,gold_none_count,pred_none_count,common_non_none,all_keys_count
884,196044,{'visit motivation': 'Inability to flex left e...,{'visit motivation': 'Inability to flex left e...,0.612640,0.967639,9.480000,0.911776,0.811710,0.901959,0.901959,1.434964,0,0,34,0,5,25,64
229,194364,{'visit motivation': 'Abnormal vaginal bleedin...,{'visit motivation': 'Abnormal vaginal bleedin...,0.449437,0.981829,7.931034,0.930881,0.726244,0.927747,0.927747,1.365167,0,0,47,0,0,29,76
444,199095,{'visit motivation': 'Complaining of back pain...,{'visit motivation': 'Complaining of back pain...,0.461334,0.989777,7.153846,0.945359,0.830744,0.939692,0.939692,1.350550,0,0,33,2,2,39,76
223,205965,"{'visit motivation': 'Cough', 'admission': [{'...","{'visit motivation': 'Cough', 'admission': [{'...",0.491301,0.975030,8.296296,0.904877,0.690638,0.904877,0.904877,1.328227,0,0,36,0,1,27,64
778,200118,{'visit motivation': 'Sustained a ground level...,{'visit motivation': 'Sustained a ground level...,0.481442,0.970755,9.148936,0.878252,0.771197,0.875729,0.875729,1.317098,3,0,31,0,1,47,82
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
617,123492,{'visit motivation': 'Dizziness and near synco...,{},NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,83,0,0,0,0,0,83
133,132207,{'visit motivation': 'Pain and swelling on rig...,{},NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,79,0,0,0,0,0,79
290,154957,{'visit motivation': 'Jaundice and dark urine'...,{},NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,103,0,0,0,0,0,103
587,2397,{'visit motivation': 'Pain on the lower right ...,{},NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,61,0,0,0,0,0,61


,keys,bleu,bert,gpt_score,rouge1,rouge2,rougeL,rougeLsum,gold_none_prop,pred_none_prop,missing_keys_prop
0,admission/0/care center details,0.270086,0.918273,7.611111,0.735385,0.650588,0.726991,0.726991,0.048,0.153,0.012
1,admission/0/date,0.231097,0.919139,7.827957,0.761162,0.608599,0.754994,0.754994,0.020,0.095,0.012
2,admission/0/duration,0.116882,0.927714,8.325843,0.762572,0.690028,0.762572,0.762572,0.021,0.049,0.012
3,admission/0/reason,0.262550,0.884549,7.142216,0.552512,0.429219,0.531419,0.531419,0.038,0.069,0.012
4,admission/1/care center details,0.082227,0.941237,8.200000,0.795455,0.666667,0.795455,0.795455,0.001,0.003,0.010
...,...,...,...,...,...,...,...,...,...,...,...
223,treatments/5/reaction to treatment,0.000000,0.926654,8.000000,0.533333,0.461538,0.533333,0.533333,0.000,0.000,0.016
224,treatments/5/reason for taking,0.333333,0.809110,4.666667,0.416667,0.333333,0.416667,0.416667,0.000,0.000,0.016
225,treatments/5/related condition,0.000000,0.833170,5.500000,0.500000,0.000000,0.500000,0.500000,0.000,0.000,0.016
226,treatments/5/time,0.000000,0.659807,1.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000,0.016


### Evaluation meditron-13b-summarizer

In [48]:
summarizer13_eval_by_sample , summarizer13_eval_by_key = summary_evaluation('meditron-13b-summarizer')

Running summary evaluation with score types: ['bleu', 'rouge', 'bert', 'gpt_score']
Loading data...
Evaluation directory exists, loading...
Progress: {'create eval directory': 'done', 'flatten and match dicts': 'done', 'clean dicts and counts': 'done', 'summary_statistics': 'done', 'scores': 'done', 'pairs_idx': 'done', 'eval_by_sample': 'tbd', 'eval_by_key': 'done', 'bleu': 'done', 'rouge': 'done', 'bert': 'done', 'gpt_score': 'done'}


In [57]:
display(summarizer13_eval_by_sample.sort_values(by='aggregated_score', ascending=False))
display(summarizer13_eval_by_key)

,idx,gold,pred,mean_bleu,mean_bert,mean_gpt_score,mean_rouge1,mean_rouge2,mean_rougeL,mean_rougeLsum,aggregated_score,missing_keys_count,extra_keys_count,common_none_count,gold_none_count,pred_none_count,common_non_none,all_keys_count
676,183398,{'visit motivation': 'Complaining of atraumati...,{'visit motivation': 'Complaining of atraumati...,0.264289,0.982445,9.291667,0.931441,0.735302,0.925330,0.925330,1.476830,0,0,38,2,0,24,64
785,187092,{'visit motivation': 'Suspicion of a complex f...,{'visit motivation': 'Suspicion of a complex f...,0.407349,0.976871,9.625000,0.873779,0.686607,0.872254,0.872254,1.371129,0,0,42,5,1,16,64
492,174469,{'visit motivation': 'Severe aortic valve insu...,{'visit motivation': 'Severe aortic valve insu...,0.503416,0.957767,8.677419,0.828210,0.737594,0.814497,0.814497,1.363922,0,0,31,1,4,31,67
670,181391,{'visit motivation': 'Painful enlargement of g...,{'visit motivation': 'Painful enlargement of g...,0.420672,0.958829,9.375000,0.824622,0.688760,0.822307,0.822307,1.361022,0,0,49,1,2,24,76
711,172876,"{'visit motivation': 'Lower leg pain', 'admiss...",{'visit motivation': 'Pain in the lower leg wh...,0.483461,0.947870,8.512821,0.842248,0.675151,0.837120,0.837120,1.354624,0,0,35,1,3,39,78
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
290,154957,{'visit motivation': 'Jaundice and dark urine'...,{},NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,103,0,0,0,0,0,103
333,63216,{'visit motivation': 'Diffuse swelling of the ...,"{'visit motivation': 1, 'admission': 1}",0.000000,0.544513,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,66,1,0,0,0,1,68
530,62901,{'visit motivation': 'Incidentally detected mi...,{},NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,61,0,0,0,0,0,61
805,52461,{'visit motivation': 'hypersexuality after sta...,{},NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,58,0,0,0,0,0,58


,keys,bleu,bert,gpt_score,rouge1,rouge2,rougeL,rougeLsum,gold_none_prop,pred_none_prop,missing_keys_prop
0,admission/0/care center details,0.264811,0.920767,7.886364,0.733560,0.643519,0.728607,0.728607,0.050,0.167,0.014
1,admission/0/date,0.222130,0.915933,7.511111,0.744379,0.559630,0.735311,0.735311,0.019,0.097,0.014
2,admission/0/duration,0.121161,0.941716,8.843373,0.808594,0.726863,0.806249,0.806249,0.019,0.054,0.014
3,admission/0/reason,0.255464,0.885055,6.565868,0.553590,0.427894,0.533150,0.533150,0.050,0.067,0.014
4,admission/1/care center details,0.160790,0.945127,8.600000,0.815455,0.777778,0.815455,0.815455,0.001,0.003,0.009
...,...,...,...,...,...,...,...,...,...,...,...
221,treatments/4/time,0.000000,0.815932,3.000000,0.272727,0.200000,0.272727,0.272727,0.000,0.000,0.026
222,treatments/5/name,0.000000,0.933010,5.500000,0.750000,0.500000,0.750000,0.750000,0.000,0.000,0.017
223,treatments/5/reason for taking,0.762368,0.993797,9.500000,0.900000,0.807692,0.900000,0.900000,0.000,0.000,0.017
224,treatments/5/related condition,0.000000,0.950773,9.000000,0.857143,0.400000,0.571429,0.571429,0.000,0.000,0.017


### Evaluation meditron-7b-direct-trunc

In [58]:
meditron_7b_direct_trunc_eval_res = note_evaluation('meditron-7b-direct-trunc')

Evaluation directory exists, loading...
Progress: {'create eval directory': 'done', 'scores': 'done', 'bleu': 'done', 'rouge': 'done', 'bert': 'done', 'gpt_score': 'done'}


In [61]:
display(meditron_7b_direct_trunc_eval_res.sort_values(by='aggregated_score', ascending=False))

,idx,gold,pred,model_name,bleu,bert,gpt_score,rouge1,rouge2,rougeL,rougeLsum,aggregated_score
251,131606,Chief complaints: A 72-year-old man was referr...,A 45-year-old male was referred to our departm...,meditron-7b-direct-trunc,0.715163,0.981494,9,0.984456,0.973958,0.984456,0.984456,1.552715
904,135418,A 44-year-old male was brought with a metallic...,A 28-year-old male patient presented with a fr...,meditron-7b-direct-trunc,0.451109,0.996472,10,0.987654,0.977667,0.987654,0.987654,1.537368
334,143164,A 16-year-old boy was brought to our psychiatr...,A 13-year-old boy was admitted to the psychiat...,meditron-7b-direct-trunc,0.864294,0.978218,9,0.942529,0.905312,0.942529,0.942529,1.519543
610,91002,A 36-year-old male sustained fracture of first...,"A 24-year-old Caucasian man, who was a paraple...",meditron-7b-direct-trunc,0.866014,0.981152,8,0.958974,0.922680,0.958974,0.958974,1.507290
840,174469,A 74-year-old man was referred to our hospital...,A 74-year-old man was referred to our hospital...,meditron-7b-direct-trunc,0.919281,0.993514,4,0.992771,0.983051,0.992771,0.992771,1.494670
...,...,...,...,...,...,...,...,...,...,...,...,...
572,78761,Sixty-three-year-old female patient was admitt...,A 57-year-old female patient was admitted to o...,meditron-7b-direct-trunc,0.000000,0.000000,6,0.000000,0.000000,0.000000,0.000000,0.077160
133,95166,A well-built 29-year-old Caucasian man attende...,A 28-year-old male presented to the emergency ...,meditron-7b-direct-trunc,0.524913,0.000000,2,0.000000,0.000000,0.000000,0.000000,0.076366
287,48732,"A 37-year-old woman, gravida 1-para-0 (BMI = 2...","A 30-year-old woman (1.65 m, 85 kg, BMI 31.2) ...",meditron-7b-direct-trunc,0.347458,0.000000,2,0.000000,0.000000,0.000000,0.000000,0.035195
653,86018,A 75 year old female reported to our clinic wi...,A 65-year old female patient presented to the ...,meditron-7b-direct-trunc,0.327465,0.000000,1,0.000000,0.000000,0.000000,0.000000,0.028519


### Evaluation meditron-13b-direct-trunc

In [60]:
meditron_13b_direct_trunc_eval_res = note_evaluation('meditron-13b-direct-trunc')

Evaluation directory exists, loading...
Progress: {'create eval directory': 'done', 'scores': 'done', 'bleu': 'done', 'rouge': 'done', 'bert': 'done', 'gpt_score': 'done'}


In [16]:
display(meditron_7b_direct_trunc_eval_res.sort_values(by='aggregated_score', ascending=False))

,idx,gold,pred,model_name,bleu,rouge1,rouge2,rougeL,rougeLsum,bert,gpt_score
0,66949,Our patient was a 60-year-old woman who had un...,A 67-year-old woman with bilateral MG II total...,meditron-7b-direct-trunc,0.328759,0.877506,0.769575,0.824053,0.824053,0.931299,7
1,163589,"A 15-year-old right-handed boy, accompanied by...",A 14-year-old boy with a complaint of pain in ...,meditron-7b-direct-trunc,0.566582,0.811736,0.663391,0.777506,0.777506,0.923475,6
2,7323,A 40-year-old male presented to our office in ...,A 40-year-old male with a medical history of d...,meditron-7b-direct-trunc,0.685503,0.626316,0.386243,0.547368,0.547368,0.875226,7
3,112273,A 12-year-old boy presented to the emergency d...,A 13-year-old boy visited the emergency depart...,meditron-7b-direct-trunc,0.520075,0.735363,0.512941,0.637002,0.637002,0.901952,9
4,46504,A 20-year-old Chinese woman first visited our ...,"A 24-year-old female, 28 weeks pregnant, was a...",meditron-7b-direct-trunc,0.575684,0.939948,0.824147,0.903394,0.903394,0.954584,6
...,...,...,...,...,...,...,...,...,...,...,...
995,90258,A 60-year-old man underwent ascending aorta re...,A 67-year-old male patient underwent ascending...,meditron-7b-direct-trunc,0.475142,0.739659,0.669927,0.715328,0.715328,0.935258,7
996,89625,A 70 year-old male presented with a 4-month hi...,A 70-year-old Caucasian man presented with a 4...,meditron-7b-direct-trunc,0.401471,0.751918,0.560411,0.700767,0.700767,0.899090,6
997,154974,An 11-year-old female patient visited the opht...,A 12-year-old girl with a four-month history o...,meditron-7b-direct-trunc,0.445235,0.396846,0.329381,0.354796,0.354796,0.925793,8
998,156151,A 28-year-old male patient reported to the Den...,A 42-year-old male patient reported with pain ...,meditron-7b-direct-trunc,0.151432,0.786408,0.600000,0.713592,0.713592,0.916245,7


### Evaluation meditron-7b-generator

In [62]:
meditron_7b_generator_eval_res = note_evaluation('meditron-7b-generator')

Evaluation directory exists, loading...
Progress: {'create eval directory': 'done', 'scores': 'done', 'bleu': 'done', 'rouge': 'done', 'bert': 'done', 'gpt_score': 'done'}


In [70]:
display(meditron_7b_generator_eval_res.sort_values(by='aggregated_score', ascending=False))

,idx,gold,pred,model_name,bleu,bert,gpt_score,rouge1,rouge2,rougeL,rougeLsum,aggregated_score
748,184746,We present the case of a 45-year-old Caucasian...,We present the case of a 47-year-old man who h...,meditron-7b-generator,0.632372,0.941218,9,0.823529,0.694087,0.757033,0.757033,1.638114
171,26362,A 24-year-old female patient was admitted to t...,A 49-year-old previously healthy man presented...,meditron-7b-generator,0.678810,0.941771,8,0.797235,0.694444,0.760369,0.760369,1.601327
214,94534,A two-and-a-half-year-old Caucasian boy with p...,A 2-year-old boy was referred to our center as...,meditron-7b-generator,0.592812,0.934824,7,0.822115,0.695652,0.764423,0.764423,1.496091
494,93075,"A 38-year-old Hispanic male from Mexico, with ...",A 55-year-old man was admitted to our hospital...,meditron-7b-generator,0.637925,0.941360,3,0.826966,0.731377,0.795506,0.795506,1.485377
633,86081,A 5-year-old girl presented to the Department ...,A 14-year-old girl presented to our clinic wit...,meditron-7b-generator,0.599475,0.926569,8,0.752747,0.674033,0.747253,0.747253,1.452073
...,...,...,...,...,...,...,...,...,...,...,...,...
537,197645,A 45-year-old woman was admitted to an emergen...,A 50-year-old female patient was admitted to o...,meditron-7b-generator,0.000000,0.765140,1,0.075188,0.005038,0.055138,0.055138,0.002871
650,74195,A three-month-old full term male was brought t...,A five-month-old male presented to the emergen...,meditron-7b-generator,0.000000,0.759605,1,0.113208,0.014218,0.066038,0.066038,0.002496
660,160722,A 62-years-old man with a palpable mass in the...,A 70-year-old male presented with a palpable m...,meditron-7b-generator,0.000000,0.761785,1,0.091787,0.009709,0.048309,0.048309,0.002130
188,187895,A 32-year old white man presented to our depar...,A 37-year old white man presented to our depar...,meditron-7b-generator,0.000000,0.756193,1,0.108911,0.014925,0.074257,0.074257,0.002086


In [74]:
with_top10_summaries = meditron_7b_generator_eval_res[meditron_7b_generator_eval_res['idx'].isin(top_10_idxs)]
display(with_top10_summaries.sort_values(by='aggregated_score', ascending=False))

,idx,gold,pred,model_name,bleu,bert,gpt_score,rouge1,rouge2,rougeL,rougeLsum,aggregated_score
312,194364,CW is a 44-year-old female with four living ch...,CW is a 44-year-old female with four living ch...,meditron-7b-generator,0.359494,0.896843,9,0.668435,0.453333,0.541114,0.541114,0.937791
219,183398,Patient is a 73 year-old right hand dominant f...,A 73-year-old right hand dominant female prese...,meditron-7b-generator,0.308846,0.884433,8,0.668317,0.472637,0.490099,0.490099,0.803176
732,191943,A 15-year-old female patient was first seen in...,A 15-year-old female patient was first seen in...,meditron-7b-generator,0.356707,0.893410,7,0.685864,0.468421,0.497382,0.497382,0.799610
593,199095,A 59-year-old African American male with a med...,A 58-year-old African American male with a med...,meditron-7b-generator,0.284445,0.877066,8,0.563969,0.351706,0.485640,0.485640,0.682328
696,196044,A 16-year-old male complained of inability to ...,A 16-year-old male complained of inability to ...,meditron-7b-generator,0.256640,0.869852,7,0.554404,0.338542,0.466321,0.466321,0.588306
341,205965,A 60-year-old man was admitted with cough. Che...,A 60-year-old man was admitted with cough. Che...,meditron-7b-generator,0.190040,0.859144,8,0.505435,0.278689,0.331522,0.331522,0.484874
535,176346,We present a case involving a 76-year-old fema...,We present a case involving a 76-year-old fema...,meditron-7b-generator,0.175769,0.858374,8,0.519048,0.258373,0.300000,0.300000,0.466003
719,187092,We report the case of a 31-year-old Caucasian ...,"We report the case of a 31-year-old woman, gra...",meditron-7b-generator,0.206926,0.845256,2,0.507177,0.302885,0.253589,0.253589,0.254923
913,170456,A 45-year-old male presented to our hospital w...,A 45-year-old male with a past medical history...,meditron-7b-generator,0.055029,0.824954,6,0.425876,0.162602,0.258760,0.258760,0.246947
933,200118,A 60-year-old female with osteoarthritis of th...,A 60-year-old female with osteoarthritis of th...,meditron-7b-generator,0.080304,0.812031,2,0.346341,0.166667,0.224390,0.224390,0.109591


### Evaluation meditron-13b-generator

In [5]:
meditron_13b_generator_eval_res = note_evaluation('meditron-13b-generator')

Evaluation directory exists, loading...
Progress: {'create eval directory': 'done', 'scores': 'done', 'bleu': 'done', 'rouge': 'done', 'bert': 'done', 'gpt_score': 'done'}


In [6]:
display(meditron_13b_generator_eval_res.sort_values(by='aggregated_score', ascending=False))

,idx,gold,pred,model_name,bleu,bert,gpt_score,rouge1,rouge2,rougeL,rougeLsum,aggregated_score
496,114194,A 26-year-old male patient was suffering from ...,A 32-year-old male patient was admitted to our...,meditron-13b-generator,0.650960,0.952268,9,0.892308,0.737113,0.861538,0.861538,1.750000
568,131669,Case 1: an 81-year-old man was taken to our ho...,A 72-year-old man took medications for osteopo...,meditron-13b-generator,0.500491,0.926586,7,0.824096,0.648910,0.761446,0.761446,1.277325
406,108424,A 25-year-old man with history of a large RAT ...,A 55-year-old female was referred for preopera...,meditron-13b-generator,0.541004,0.918477,8,0.762115,0.650442,0.704846,0.704846,1.248709
816,52583,A 41-year-old female patient presented with a ...,A 32-year-old male patient presented with left...,meditron-13b-generator,0.513387,0.913561,8,0.764846,0.596659,0.655582,0.655582,1.146794
995,34637,A four-year old Iranian girl was referred to t...,A five-year-old female child was admitted to o...,meditron-13b-generator,0.514928,0.904675,9,0.717172,0.558376,0.631313,0.631313,1.125451
...,...,...,...,...,...,...,...,...,...,...,...,...
98,183830,The patient was a 31 y/o Gravida 3 Para 2 who ...,A 31 year old Gravida 3 Para 2 presented at 19...,meditron-13b-generator,0.000000,0.751483,1,0.096203,0.010178,0.055696,0.055696,0.004593
786,170531,We report a case of 38-year-old Greek woman wh...,A 55-year-old female patient was admitted to o...,meditron-13b-generator,0.021377,0.743286,1,0.109091,0.020888,0.057143,0.057143,0.003923
920,176459,"In June 2011, a 56-year-old male was referred ...","In May 2005, a 63-year-old male was diagnosed ...",meditron-13b-generator,0.000000,0.736250,1,0.077720,0.005208,0.051813,0.051813,0.001322
178,162933,A 40-year-old female presented to orthopedics-...,A 60-year-old female presented to the out-pati...,meditron-13b-generator,0.000000,0.726798,1,0.047120,0.005263,0.041885,0.041885,0.000235


### Evaluation gpt3-direct

In [10]:
gpt3_direct_eval_res = note_evaluation('gpt3-direct')

Creating Evaluation directory and progress monitoring
Removing stopwords...


100%|██████████| 1000/1000 [00:05<00:00, 174.39it/s]


Stopwords removed.


Computing Rouge scores: 100%|██████████| 10/10 [02:51<00:00, 17.14s/it]


Computing BERTscores...


Computing BERT scores: 100%|██████████| 10/10 [18:01<00:00, 108.18s/it]


GPT-4 scoring...


Building sub-batches: 100%|██████████| 67/67 [00:02<00:00, 29.21it/s]


Total input tokens: 775836, total input cost: 7.758360000000001$
Sub_batch 1/2: 42 calls, 491839 total tokens: 4.9183900000000005$
....................20
....................40
..

Break for 5 seconds.
End of break.
Sub_batch 2/2: 25 calls, 283997 total tokens: 2.83997$
....................20
.....

Break for 5 seconds.
End of break.


In [11]:
display(gpt3_direct_eval_res.sort_values(by='aggregated_score', ascending=False))

,idx,gold,pred,model_name,bleu,rouge1,rouge2,rougeL,rougeLsum,bert,gpt_score,aggregated_score
465,134359,A 50-year-old female presented to us with a 3-...,Subjective:\nThe patient presented with a chie...,gpt3-direct,0.598169,0.838852,0.740576,0.825607,0.825607,0.933021,9,1.628411
625,149538,A 30-year-old African American male developed ...,Patient presented to the office with a history...,gpt3-direct,0.613840,0.838407,0.672941,0.786885,0.786885,0.942379,9,1.568258
565,181391,A 70-year-old Sri Lankan Tamil male with a his...,"Mr. Jones, a 55-year-old male with a history o...",gpt3-direct,0.592184,0.857143,0.712963,0.806452,0.806452,0.950346,7,1.545642
364,86402,A 27-year-old female patient with good general...,Patient presented to the endodontic department...,gpt3-direct,0.551883,0.859375,0.743455,0.796875,0.796875,0.920314,9,1.543260
742,125854,"A healthy, athletic 19-year-old African Americ...",Patient presented to the clinic for a surgical...,gpt3-direct,0.575554,0.861386,0.706468,0.792079,0.792079,0.939554,8,1.527324
...,...,...,...,...,...,...,...,...,...,...,...,...
363,186100,A 24-year-old female patient presented to the ...,Patient presented with a suspicious upper lip ...,gpt3-direct,0.039935,0.356643,0.176056,0.216783,0.216783,0.818662,3,0.110423
691,121545,An 8-month-old girl presented in the pediatric...,Patient presents with a chief complaint of vom...,gpt3-direct,0.000567,0.200000,0.112903,0.184000,0.184000,0.814555,1,0.051937
245,152597,The second case was a 70-year-old married woma...,Patient presented today with complaints of fee...,gpt3-direct,0.000000,0.229249,0.023904,0.126482,0.126482,0.780612,3,0.033704
412,132207,A 45-year-old female patient reported in Augus...,Date: [Date of visit]\nPatient Name: [Patient'...,gpt3-direct,0.003428,0.143460,0.068085,0.118143,0.118143,0.746065,2,0.009222


### Evaluation mistral-7b-direct

In [5]:
mistral_7_b_eval_res = note_evaluation('mistral-7b-direct')

Evaluation directory exists, loading...
Progress: {'create eval directory': 'done', 'scores': 'tbd', 'bleu': 'done', 'rouge': 'done', 'bert': 'in progress', 'gpt_score': 'tbd'}
Removing stopwords...


100%|██████████| 1000/1000 [00:04<00:00, 216.57it/s]
/Users/paul/Desktop/Make-Project/ClinicalNotes/utils/scorer.py:191: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  res['bert'] = computed['bert']


Stopwords removed.
BLEU scores already computed.
ROUGE scores already computed.
Computing BERTscores...
BERTscores computation already in progress, resuming


Computing BERT scores: 100%|██████████| 4/4 [43:09<00:00, 647.45s/it] 


GPT-4 scoring...


Building sub-batches: 100%|██████████| 67/67 [00:02<00:00, 29.01it/s]


Total input tokens: 775429, total input cost: 7.75429$
Sub_batch 1/2: 43 calls, 499542 total tokens: 4.99542$
....................20
....................40
...

Break for 5 seconds.
End of break.
Sub_batch 2/2: 24 calls, 275887 total tokens: 2.75887$
....................20
....

Break for 5 seconds.
End of break.


In [6]:
display(mistral_7_b_eval_res.sort_values(by='aggregated_score', ascending=False))

,idx,gold,pred,model_name,bleu,rouge1,rouge2,rougeL,rougeLsum,bert,gpt_score,aggregated_score
93,89625,A 70 year-old male presented with a 4-month hi...,\nPatient: \nName: [REDACTED]\nAge: [REDACTED]...,mistral-7b-direct,0.446003,0.841346,0.700483,0.802885,0.802885,0.918869,8,1.512481
689,189138,"A 15-year-old boy, grade 9 student, presented ...",\nPatient: [Name]\nAge: [Age]\nSex: [Sex]\nMed...,mistral-7b-direct,0.550962,0.808399,0.670185,0.766404,0.766404,0.867168,8,1.436817
302,99890,A 69-year-old male presented to the emergency ...,\nPatient: 65-year-old male with a history of ...,mistral-7b-direct,0.437850,0.817708,0.628272,0.760417,0.760417,0.919878,8,1.398805
452,194450,A 54-year-old Caucasian female proceeded to ou...,\nPatient: [Name]\nDate of Birth: [Date]\nSex:...,mistral-7b-direct,0.489894,0.837529,0.735632,0.810069,0.810069,0.899905,2,1.397145
424,107887,A 53-year-old male pedestrian was hit by a car...,"\nPatient: [Name], a 45-year-old male, present...",mistral-7b-direct,0.458563,0.784119,0.588529,0.759305,0.759305,0.917910,8,1.367926
...,...,...,...,...,...,...,...,...,...,...,...,...
142,132386,A 2-day-old infant girl was referred to emerge...,"\nPatient: [Name], a 2-year-old female, presen...",mistral-7b-direct,0.010432,0.192469,0.109705,0.108787,0.108787,0.757922,2,0.066694
559,156009,A 32-year-old man with previous healthy condit...,\nPatient: \nName: [REDACTED]\nAge: [REDACTED]...,mistral-7b-direct,0.000000,0.186992,0.081967,0.081301,0.081301,0.764138,1,0.061061
248,202224,A 48-year-old Caucasian man presented to the N...,\nPatient: A 45-year-old female with a history...,mistral-7b-direct,0.000000,0.162011,0.033708,0.094972,0.094972,0.768188,1,0.060184
100,192066,A 55-year-old man with a history of extensive ...,\nPatient: [Name]\nDOB: [Date of Birth]\nSex: ...,mistral-7b-direct,0.000000,0.019481,0.000000,0.019481,0.019481,0.688761,1,0.005445


### Evaluation gpt3-generator-gpt

In [11]:
gpt3_generator_gpt_eval_res = note_evaluation('gpt3-generator-gpt')

Evaluation directory exists, loading...
Progress: {'create eval directory': 'done', 'scores': 'tbd'}
Removing stopwords...


100%|██████████| 1000/1000 [00:06<00:00, 151.53it/s]


Stopwords removed.


Computing Rouge scores: 100%|██████████| 10/10 [03:17<00:00, 19.77s/it]


Computing BERTscores...


Computing BERT scores: 100%|██████████| 10/10 [23:37<00:00, 141.72s/it]


GPT-4 scoring...


Building sub-batches: 100%|██████████| 67/67 [00:01<00:00, 50.64it/s]


Total input tokens: 860112, total input cost: 8.60112$
Sub_batch 1/2: 38 calls, 491628 total tokens: 4.9162799999999995$
....................20
..................

Break for 5 seconds.
End of break.
Sub_batch 2/2: 29 calls, 368484 total tokens: 3.68484$
....................20
.........

Break for 5 seconds.
End of break.


In [12]:
display(gpt3_generator_gpt_eval_res.sort_values(by='aggregated_score', ascending=False))

,idx,gold,pred,model_name,bleu,rouge1,rouge2,rougeL,rougeLsum,bert,gpt_score,aggregated_score
552,134359,A 50-year-old female presented to us with a 3-...,Patient is a 50-year-old female who presented ...,gpt3-generator-gpt,0.548937,0.837321,0.721154,0.794258,0.794258,0.909886,9,1.647630
781,115484,A 71-year-old female patient was referred to t...,A 71-year-old female patient presented to the ...,gpt3-generator-gpt,0.525842,0.784119,0.648379,0.694789,0.694789,0.921207,9,1.436649
202,181391,A 70-year-old Sri Lankan Tamil male with a his...,The patient is a 70-year-old male of Sri Lanka...,gpt3-generator-gpt,0.537672,0.789216,0.625616,0.573529,0.573529,0.918851,9,1.293760
305,79169,We report the case of a 27-year-old Tunisian m...,A 27-year-old male patient presented with dysp...,gpt3-generator-gpt,0.444729,0.771242,0.568928,0.631808,0.631808,0.917579,9,1.221349
996,71990,An 81-year-old female with medical history sig...,Patient Summary:\nThis is an 81-year-old femal...,gpt3-generator-gpt,0.444621,0.761021,0.545455,0.631090,0.631090,0.923434,9,1.219362
...,...,...,...,...,...,...,...,...,...,...,...,...
948,152597,The second case was a 70-year-old married woma...,Clinical Note:\n\nPatient presented with sympt...,gpt3-generator-gpt,0.079810,0.306748,0.123457,0.184049,0.184049,0.792763,8,0.164342
825,115691,The donor was a 16-month old male child (blood...,Clinical Note - Organ Donation after Brain Dea...,gpt3-generator-gpt,0.105336,0.365688,0.213152,0.216704,0.216704,0.813712,7,0.164202
467,108437,"A 16-year-old female, presented with problems ...",Clinical Note:\n\nThis is a 16-year-old female...,gpt3-generator-gpt,0.115957,0.391437,0.203077,0.220183,0.220183,0.804913,5,0.101765
10,28369,B is a 59 year-old man living in Rome with his...,Clinical Note:\n\nThis is a clinical note for ...,gpt3-generator-gpt,0.038661,0.170213,0.085714,0.099291,0.099291,0.775800,7,0.097656


### Evaluation gpt3-generator-7b

In [15]:
gpt3_generator_7b = note_evaluation('gpt3-generator-7b')

Creating Evaluation directory and progress monitoring
Removing stopwords...


100%|██████████| 1000/1000 [00:07<00:00, 133.87it/s]


Stopwords removed.


Computing Rouge scores: 100%|██████████| 10/10 [03:24<00:00, 20.49s/it]


Computing BERTscores...


Computing BERT scores: 100%|██████████| 10/10 [26:46<00:00, 160.63s/it]


GPT-4 scoring...


Building sub-batches: 100%|██████████| 67/67 [00:03<00:00, 21.58it/s]


Total input tokens: 854713, total input cost: 8.54713$
Sub_batch 1/2: 39 calls, 497627 total tokens: 4.97627$
....................20
...................

Break for 5 seconds.
End of break.
Sub_batch 2/2: 28 calls, 357086 total tokens: 3.57086$

Exception occurred: Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4-1106-preview in organization org-tUY6bwaZRR8IwltlIGVE2rVA on tokens per min (TPM): Limit 600000, Used 594429, Requested 16275. Please try again in 1.07s. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}
Retrying chat (1/5)...
....................20
........

Break for 5 seconds.
End of break.


In [16]:
display(gpt3_generator_7b.sort_values(by='aggregated_score', ascending=False))

,idx,gold,pred,model_name,bleu,rouge1,rouge2,rougeL,rougeLsum,bert,gpt_score,aggregated_score
969,134359,A 50-year-old female presented to us with a 3-...,Clinical Note:\n\nA 50-year-old female patient...,gpt3-generator-7b,0.533346,0.810934,0.691076,0.701595,0.701595,0.906316,9,1.565065
543,79169,We report the case of a 27-year-old Tunisian m...,A 27-year-old male patient of Tunisian ethnici...,gpt3-generator-7b,0.482289,0.784483,0.593074,0.711207,0.711207,0.922035,9,1.491929
660,196303,A 68-year-old man presented with a ten-year hi...,Patient is a 68-year-old male with a ten-year ...,gpt3-generator-7b,0.542131,0.788462,0.647343,0.634615,0.634615,0.930085,9,1.488304
890,196993,The patient was a 39-year-old female who had e...,A 39-year-old female presented with a graduall...,gpt3-generator-7b,0.576397,0.824121,0.661616,0.678392,0.678392,0.898149,8,1.482198
179,176303,A 68-year-old man presented with a ten-year hi...,This is a clinical note for a 68-year-old male...,gpt3-generator-7b,0.497164,0.762332,0.612613,0.636771,0.636771,0.925585,9,1.397053
...,...,...,...,...,...,...,...,...,...,...,...,...
690,195500,A 46-year-old male with a history of severe de...,I evaluated a 46-year-old male patient who pre...,gpt3-generator-7b,0.120251,0.440433,0.290909,0.259928,0.259928,0.838146,2,0.156532
894,34468,A 42-year-old male was operated for carcinoma ...,Clinical Note:\n\nPatient ID: [Patient ID]\nAg...,gpt3-generator-7b,0.092528,0.309859,0.141509,0.201878,0.201878,0.803050,7,0.150696
710,200090,Case 6: Lidocaine (Spring 2018)\nThe patient i...,Patient Name: [Name]\nAge: 70\nSex: Female\n\n...,gpt3-generator-7b,0.078897,0.211382,0.103542,0.119241,0.119241,0.776798,7,0.116868
615,73609,"Case 2. In November 2014, a woman aged 69 year...",Patient presented to the clinic with complaint...,gpt3-generator-7b,0.184122,0.375691,0.166667,0.226519,0.226519,0.829222,2,0.106567


### Evaluation gpt3-generator-13b

In [ ]:
gpt3_generator_13b_eval_res = note_evaluation('gpt3-generator-13b')

In [ ]:
display(gpt3_generator_13b_eval_res.sort_values(by='aggregated_score', ascending=False))

### Evaluation  meditron-7b-generator-gold

In [ ]:
meditron_7b_generator_gold_eval_res = note_evaluation('meditron-7b-generator-gold')

In [ ]:
display(meditron_7b_generator_gold_eval_res.sort_values(by='aggregated_score', ascending=False))

### Evaluation meditron-13b-generator-gold

In [ ]:
meditron_13b_generator_gold_eval_res = note_evaluation('meditron-13b-generator-gold')

In [ ]:
display(meditron_13b_generator_gold_eval_res.sort_values(by='aggregated_score', ascending=False))

### Merging note evaluation

In [19]:
final_eval = merge_note_evaluations(model_names= note_models)

evaluation/gpt3-generator-13b_eval_res/all_scores.jsonl not existing yet
evaluation/meditron-7b-generator-gold_eval_res/all_scores.jsonl not existing yet
evaluation/meditron-13b-generator-gold_eval_res/all_scores.jsonl not existing yet


In [20]:
display(final_eval)

,model,bleu,bert,gpt_score,rouge1,rouge2,rougeL,rougeLsum
0,meditron-7b-direct-trunc,0.439479,0.899691,6.364,0.729841,0.563817,0.665678,0.665678
1,meditron-13b-direct-trunc,0.434462,0.898802,6.533,0.729459,0.563043,0.664638,0.664638
2,meditron-7b-generator,0.251536,0.859992,5.360,0.535441,0.331078,0.386767,0.386767
3,meditron-13b-generator,0.246159,0.860041,5.345,0.537501,0.329094,0.388867,0.388867
4,mistral-7b-direct,0.223627,0.852028,6.274,0.549584,0.364188,0.441554,0.441554
5,gpt3-direct,0.235334,0.872546,8.295,0.588361,0.377739,0.498093,0.498093
6,gpt3-generator-gpt,0.249655,0.869048,8.394,0.576516,0.377367,0.388395,0.388395
7,gpt3-generator-7b,0.248659,0.869052,8.308,0.575296,0.376052,0.389251,0.389251
